# !!No Longer Used for Analysis!!

In [1]:
#!/usr/bin/env python

"""
12/26/2017  carolyn.parkinson@gmail.com

After preprocessing, this script does the following for each subject:
    1) Merges the preprocessed time series data across the 6 runs
    2) Extracts this average preprocessed time series from each anatomical ROI
    3) Saves the average preprocessed time series for each ROI in a dictionary
    where the keys are the ROI labels and the values are the corresponding time
    series.
"""

# import relevant python modules
import os
import shutil
import subprocess
import json
import glob
import csv
import numpy as np

In [2]:
# set path to directory containing data for all subjects
mypath = ('/Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts')

# read in the list of subjects in the study whose data we'll be preprocesssing
#list_file = '{}/subjects.json'.format(mypath)
#with open(list_file) as data_file:
#    subj_list = json.load(data_file)

subj_list = ["sub-289"]
# define a function to execute shell commands
def sh(c):
    '''
    run shell commands
    '''
    subprocess.call(c, shell = True)
    #print(c)

# define a function to write dictionaries to files with json encoding
def ld_writeDicts(filePath,dict):
    f = open(filePath,'w')
    newData = json.dumps(dict, sort_keys = True, indent = 4)
    f.write(newData)
    f.close()


In [3]:
#for subject in subj_list:
subject = "sub-289"

print(mypath)
print(subject)
# specify and enter directory containing this subject's nifti files
nifti_dir = "{}/nifti/{}".format(mypath, subject)
os.chdir(nifti_dir)

# also specify directory containing their SUMA files
suma_dir = "{}/{}/SUMA".format(mypath, subject)

# copy each run of preprocessed data into the subject's main nifti directory
epi_fnames = {}
for epi_num in range(1,7):
    # specify the prefix for the preprocessed file names for this run; this is
    # also the name of the AFNI-generated sub-directory containing these files
    epi_prefix = 'epi{}_preprocessed'.format(epi_num)
    # specify file name of preprocessed data within that sub-directory
    epi_fname = '{}.cleanEPI+orig'.format(epi_prefix)
    # create a dictionary where  keys are the run names ('epi1', 'epi2', etc.)
    # and values are the corresponding full file names (to iterate through later)
    epi_fnames['epi{}'.format(epi_num)] = epi_fname
    # specify and execute an AFNI command to copy these files
    cp_cmd = '3dcopy {}/{}/{} {}/{}.cleanEPI'.format(nifti_dir, epi_prefix, epi_fname, nifti_dir, epi_prefix)
    sh(cp_cmd)

# specify what the file of concatenated data should be called
all_epi = "{}_epi_all_preprocessed".format(subject)
# specify and execute an AFNI command to concatenate all runs' preprocessed data
tcat_cmd = ("3dTcat -session {} -prefix {} "
            "{epi1} {epi2} {epi3} {epi4} {epi5} {epi6}").format(nifti_dir, all_epi,  **epi_fnames)
sh(tcat_cmd)






/Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts
sub-289


In [4]:
# Clean up directory a bit
# delete redundant copies of preprocessed data for each run
for num, fname in epi_fnames.iteritems():
    rm_cmd = 'rm {}/{}*'.format(nifti_dir, fname)
    sh(rm_cmd)

# also move raw epi data into its own directory
raw_epi_dir = '{}/raw_epi_data'.format(nifti_dir)
if not os.path.exists(raw_epi_dir):
    os.makedirs(raw_epi_dir) # create this directory if it doesn't exist already
mv_cmd = 'mv {}/{}_fMRI_* {}'.format(nifti_dir, subject, raw_epi_dir)
sh(mv_cmd)

In [12]:
    # specify table of ROI labels from Freesurfer cortical parcellation
    lookup_table = "{}/aparc+aseg_rank.niml.lt".format(suma_dir)

    # create a dictionary to fill that matches ROI numbers to labels
    roi_dict = {}

    with open(lookup_table) as csvfile:
        # skip first 4 lines (header)
        next(csvfile)
        next(csvfile)
        next(csvfile)
        next(csvfile)
        reader = csv.DictReader(csvfile, delimiter = " ", fieldnames = ["roi_num", "roi_label"])
        for row in reader:
            roi_dict[row['roi_num']] = row['roi_label']

    # save as text file with json encoding
    ld_writeDicts('{}_roi_dict_dk_atlas.json'.format(subject), roi_dict)

    # Now get masks of cortical areas from Freesurfer parcellation so that we can
    # extract the average time series from each mask.

    # Get the total number of ROIs in this subject's parcellation file; this is
    # 1 less than the length of the lookup table due to the file footer. The
    # length of different subjects' lookup tables may vary a bit, as some
    # subjects have a 5th ventricle in the FreeSurfer subcortical segmentation;
    # see http://surfer.nmr.mgh.harvard.edu/fswiki/SubcorticalSegmentation/
    all_roi_num = range(1, (len(roi_dict)-1))

    # get Freesurfer parcellation file that's been aligned to our anatomical scan
    #aparc = "{}/aparc+aseg_rank_Alnd_Exp".format(nifti_dir)
    aparc = "{}/aseg".format(nifti_dir)


    # resample parcellation file to functional resolution
    resamp_cmd = ("3dresample -master {} -rmode NN "
                  "-input {} -prefix {}_3mm").format(all_epi, aparc, aparc)
    sh(resamp_cmd)


In [10]:
    # dictionary to store preprocessed time series indexed by ROI
    subj_ts_dict = {}

    for roi_id in all_roi_num:
        # extract ROI label form ROI dictionary
        label_roi = roi_dict[str(roi_id)]

        # Extract the average time series across voxels that ROI
        avg_cmd = ("""3dmaskave -quiet -mask {}_3mm+orig"<{}..{}>" {}+orig > avg_{}.1D""").format(aparc, roi_id, roi_id, all_epi, label_roi)
        sh(avg_cmd)
        
        print(avg_cmd)

        # Then read in the 1D file generated by the above command (average time
        # series across voxels within a given ROI) and add it to a dictionary
        # indexed by ROI label
        this_vec = "avg_{}.1D".format(label_roi)
        subj_ts_dict[label_roi] = np.loadtxt(this_vec).tolist()
        
    print(subj_ts_dict)

    # save dictionary to a text file with json encoding
    ld_writeDicts('{}_roi_dk_atlas_ts_dict.json'.format(subject), subj_ts_dict)

    # move the individual ROI-wise time series 1D files to their own sub-directory
    sub_dir = "{}/avg".format(nifti_dir)
    if not os.path.exists(sub_dir):
        os.makedirs(sub_dir) # create qvecs if it doesn't exist
    mv_sub_cmd = "mv avg*1D avg/"
    sh(mv_sub_cmd)


3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<1..1>" sub-289_epi_all_preprocessed+orig > avg_Left-Cerebral-White-Matter.1D
3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<2..2>" sub-289_epi_all_preprocessed+orig > avg_Left-Lateral-Ventricle.1D
3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<3..3>" sub-289_epi_all_preprocessed+orig > avg_Left-Inf-Lat-Vent.1D
3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<4..4>" sub-289_epi_all_preprocessed+orig > avg_Left-Cerebellum-White-Matter.1D
3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<5..5>" sub-289_epi_all_preprocessed+orig > avg_Left-Cerebellum-Cortex.1D


/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_Left-Cerebral-White-Matter.1D"
/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_Left-Lateral-Ventricle.1D"
/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_Left-Inf-Lat-Vent.1D"
/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_Left-Cerebellum-White-Matter.1D"
/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_Left-Cerebellum-Cortex.1D"


3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<6..6>" sub-289_epi_all_preprocessed+orig > avg_Left-Thalamus-Proper.1D
3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<7..7>" sub-289_epi_all_preprocessed+orig > avg_Left-Caudate.1D
3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<8..8>" sub-289_epi_all_preprocessed+orig > avg_Left-Putamen.1D
3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<9..9>" sub-289_epi_all_preprocessed+orig > avg_Left-Pallidum.1D
3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<10..10>" sub-289_epi_all_preprocessed+orig > avg_3rd-Ventricle.1D


/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_Left-Thalamus-Proper.1D"
/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_Left-Caudate.1D"
/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_Left-Putamen.1D"
/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_Left-Pallidum.1D"
/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_3rd-Ventricle.1D"


3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<11..11>" sub-289_epi_all_preprocessed+orig > avg_4th-Ventricle.1D
3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<12..12>" sub-289_epi_all_preprocessed+orig > avg_Brain-Stem.1D
3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<13..13>" sub-289_epi_all_preprocessed+orig > avg_Left-Hippocampus.1D
3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<14..14>" sub-289_epi_all_preprocessed+orig > avg_Left-Amygdala.1D
3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<15..15>" sub-289_epi_all_preprocessed+orig > avg_CSF.1D


...

/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_ctx-rh-precuneus.1D"
/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_ctx-rh-rostralanteriorcingulate.1D"
/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_ctx-rh-rostralmiddlefrontal.1D"
/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_ctx-rh-superiorfrontal.1D"
/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_ctx-rh-superiorparietal.1D"


3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<106..106>" sub-289_epi_all_preprocessed+orig > avg_ctx-rh-superiortemporal.1D
3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<107..107>" sub-289_epi_all_preprocessed+orig > avg_ctx-rh-supramarginal.1D
3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<108..108>" sub-289_epi_all_preprocessed+orig > avg_ctx-rh-frontalpole.1D
3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<109..109>" sub-289_epi_all_preprocessed+orig > avg_ctx-rh-temporalpole.1D


/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_ctx-rh-superiortemporal.1D"
/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_ctx-rh-supramarginal.1D"
/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_ctx-rh-frontalpole.1D"
/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_ctx-rh-temporalpole.1D"


3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<110..110>" sub-289_epi_all_preprocessed+orig > avg_ctx-rh-transversetemporal.1D
3dmaskave -quiet -mask /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg_3mm+orig"<111..111>" sub-289_epi_all_preprocessed+orig > avg_ctx-rh-insula.1D
{'ctx-rh-precentral': [], 'ctx-lh-rostralmiddlefrontal': [], 'Left-Amygdala': [], 'ctx-lh-frontalpole': [], 'ctx-lh-caudalmiddlefrontal': [], 'ctx-rh-frontalpole': [], 'ctx-rh-parstriangularis': [], 'Left-Cerebellum-White-Matter': [], 'Right-Putamen': [], 'ctx-lh-inferiortemporal': [], 'ctx-lh-posteriorcingulate': [], 'ctx-rh-parahippocampal': [], 'ctx-lh-rostralanteriorcingulate': [], 'ctx-lh-fusiform': [], 'ctx-rh-medialorbitofrontal': [], 'ctx-rh-rostralmiddlefrontal': [], 'ctx-lh-lateraloccipital': [], 'Left-Lateral-Ventricle': [], 'ctx-rh-cuneus': [], 'ctx-rh-inferiortemporal': [], 'CC_Central': [], 'Right-Hippocampus': [], 'ctx-lh-lingual

/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_ctx-rh-transversetemporal.1D"
/Users/mdclark/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "avg_ctx-rh-insula.1D"
